1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있음.
    - 문서가 길면 답변 생성이 오래걸림
3. 임베딩 하여 백터 데이터에 저장 
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM 에 질문과 같이 전달.

## 1. 문서의 내용을 읽는다.

In [1]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax_with_markdown.docx")
document = loader.load()

len(document)

1

In [2]:
len(document[0].page_content)

290555

## 2. 문서를 쪼갠다.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=20
)

document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

207

## 3. 임베딩 하여 백터 데이터에 저장 

In [4]:
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv

load_dotenv()
embeddings = UpstageEmbeddings(model="solar-embedding-1-large") 

In [5]:
# %pip install langchain-pinecone
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index-markdown'

# Set your Pinecone API key here if not already set
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# 데이터 추가하기
# database = PineconeVectorStore.from_documents(document_list, embeddings, index_name=index_name)

# 데이터베이스가 이미 존재하는 경우
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

## 4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색

In [6]:
query = "연봉 5천만원 직장인 소득세는 얼마인가요?"
retriever = database.as_retriever(search_kwargs={"k": 1})

In [7]:
retriever.invoke(query)

[Document(id='ed75a997-4a67-424b-9a44-84ea176c12d2', metadata={'source': './tax_with_markdown.docx'}, page_content='제54조의2(공동사업에 대한 소득공제 등 특례) 제51조의3 또는 「조세특례제한법」에 따른 소득공제를 적용하거나 제59조의3에 따른 세액공제를 적용하는 경우 제43조제3항에 따라 소득금액이 주된 공동사업자의 소득금액에 합산과세되는 특수관계인이 지출ㆍ납입ㆍ투자ㆍ출자 등을 한 금액이 있으면 주된 공동사업자의 소득에 합산과세되는 소득금액의 한도에서 주된 공동사업자가 지출ㆍ납입ㆍ투자ㆍ출자 등을 한 금액으로 보아 주된 공동사업자의 합산과세되는 종합소득금액 또는 종합소득산출세액을 계산할 때에 소득공제 또는 세액공제를 받을 수 있다. <개정 2012. 1. 1., 2014. 1. 1.>\n\n[전문개정 2009. 12. 31.]\n\n[제목개정 2014. 1. 1.]\n\n\n\n제4절 세액의 계산 <개정 2009. 12. 31.>\n\n\n\n제1관 세율 <개정 2009. 12. 31.>\n\n\n\n제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하

## 5. 유사도 검색으로 가져온 문서를 LLM 에 질문과 같이 전달.

In [8]:
from langchain_upstage import ChatUpstage
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
llm = ChatUpstage()

c:\Users\owner\Desktop\coding\llm-application\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
# DB 에 as_retriever 를 사용하여 연결.
# as_retriever 는 chroma 뿐만 아니라 다양한 데이터베이스에서 사용할 수 있다.
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke(query)
print(ai_message)

{'query': '연봉 5천만원 직장인 소득세는 얼마인가요?', 'result': '연봉 5천만원의 직장인 소득세는 538만원입니다. 이는 5천만원의 10%에 해당하는 5백만원에 누진공제액 52만원을 더한 금액입니다.'}


In [10]:
# 직장인으로 입력받는 경우, 거주자로 변경하는 chain을 추가한다.

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현은 모두 거주자로 변경해주세요"]
prompt = ChatPromptTemplate.from_template(f"""
                                사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
                                변경할 필요가 없다고 판단되면, 사용자 질문을 변경하지 않아도 됩니다.
                                그런 경우에는 질문만 return 해주세요.
                                사전: {dictionary}
                                질문 : {query}
                                """)

dictionary_chain = prompt | llm | StrOutputParser()


In [11]:
# 수정 코드 (정상 동작)
new_question = dictionary_chain.invoke({"query": query})
print(query)
print(new_question)

연봉 5천만원 직장인 소득세는 얼마인가요?
연봉 5천만원 거주자 소득세는 얼마인가요?


In [12]:
tax_chain = {"query": dictionary_chain} | qa_chain
ai_response = tax_chain.invoke({"question" : query})
ai_response

{'query': '연봉 5천만원 거주자 소득세는 얼마인가요?',
 'result': "연봉 5천만원의 경우, 소득세 과세표준에 따라 세율을 적용하면 됩니다. 5천만원은 '1,400만원 초과 5,000만원 이하' 구간에 해당하여, 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)를 계산하면 됩니다. 따라서, 연봉 5천만원 거주자의 소득세는 5,100,000원입니다."}

{'query': '연봉 5천만원 거주자 소득세는 얼마인가요?',
 'result': '연봉 5천만원의 경우, 종합소득 과세표준은 5천만원 초과 8,800만원 이하 구간에 해당합니다. 따라서 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)를 적용하여 계산한 종합소득산출세액을 내야 합니다. 구체적인 계산은 다음과 같습니다. 5천만원에서 5천만원을 초과하는 금액은 0이므로, 624만원이 종합소득산출세액이 됩니다. 그러나 근로소득공제와 세액공제가 적용될 수 있으므로, 정확한 세액은 공제 내역을 확인한 후에 계산해야 합니다.'}